<a href="https://colab.research.google.com/github/R1A2H1L1/KaggleProject/blob/main/04%20-%20modelo%20con%20preprocesado%20A%20y%20Regresi%C3%B3n%20Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
!pip install Unidecode
import unidecode
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from progressbar import progressbar as pbar
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.0 MB/s eta 0:00:00


In [3]:
students_df = pd.read_csv("train.csv")

In [4]:
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

In [6]:
# Las funciones también fueron extraidas de los Colabs compartidos por el profesor
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

columns_to_onehot =[
    'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)
onehoted_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Técnica o tecnológica incompleta,Si,Si,N,...,0.322,0.208,0.31,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,Técnica o tecnológica completa,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,No sabe,Si,No,N,...,0.485,0.172,0.252,0.19,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Primaria completa,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212224,20212,INGENIERÍA INDUSTRIAL,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 2,Secundaria (Bachillerato) completa,Si,No,N,...,0.233,0.284,0.298,0.311,0,0,1,0,0,1
212225,20183,INGENIERIA AMBIENTAL,SANTANDER,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 3,Primaria completa,Si,No,N,...,0.315,0.214,0.318,0.267,0,0,1,0,1,0
212226,20203,MEDICINA,QUINDIO,Más de 7 millones,Más de 30 horas,Estrato 5,Educación profesional completa,Si,Si,N,...,0.529,0.126,0.249,0.187,0,0,1,0,1,0
212227,20195,DERECHO,VALLE,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 4,Primaria incompleta,Si,Si,N,...,0.469,0.155,0.265,0.193,0,0,1,0,0,1


In [7]:
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

In [9]:
data_matricula_dict = {
    'Miss': 0,
    'No pagó matrícula': 1,
    'Menos de 500 mil': 2,
    'Entre 500 mil y menos de 1 millón': 3,
    'Entre 1 millón y menos de 2.5 millones': 4,
    'Entre 2.5 millones y menos de 4 millones': 5,
    'Entre 4 millones y menos de 5.5 millones': 6,
    'Entre 5.5 millones y menos de 7 millones': 7,
    'Más de 7 millones': 8,
 }

data_working_hours_dict = {
    'Miss': 0,
    '0': 1,
    'Menos de 10 horas': 2,
    'Entre 11 y 20 horas': 3,
    'Entre 21 y 30 horas': 4,
    'Más de 30 horas': 5,
 }

data_estrato_dict = {
    'Miss': -1,
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
 }

data_education_dict = {
    'Miss': -2,
    'No Aplica': -1,
    'Ninguno': 0,
    'No sabe': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 4,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 6,
    'Técnica o tecnológica completa': 7,
    'Educación profesional incompleta': 8,
    'Educación profesional completa': 9,
    'Postgrado': 10,
 }

data_performance_dict = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
 }

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "E_VALORMATRICULAUNIVERSIDAD",
    data_matricula_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "E_HORASSEMANATRABAJA",
    data_working_hours_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "F_ESTRATOVIVIENDA",
    data_estrato_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "F_EDUCACIONPADRE",
    data_education_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "F_EDUCACIONMADRE",
    data_education_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "RENDIMIENTO_GLOBAL",
    data_performance_dict)

onehoted_df.head()

/tmp/ipython-input-3900146150.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,7,2,3,6,Si,Si,N,...,0.322,0.208,0.31,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,5,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.19,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0


In [10]:
def replace_word(word, sentence: str):
  return sentence.replace(" DE ", " ")

def replace_wrong_words(sentence):
  result = sentence
  wrong_words_dict = {
      '3DEG ': '',
      ' PRIMARIA ': ' ', # Hay ciertas carreras que tienen esta palabra extra
      ' PRIMARIA:': ' ',
      'INGENIER?A': 'INGENIERIA',
      'II': 'I',
      'EDUCACI?N': 'EDUCACION',
      'F?SICA': 'FISICA',
      'RECREACI?N': 'RECREACION',
      '?NFASIS': 'ENFASIS',
      'INGL?S': 'INGLES',
      'DISE?O': 'DISENO',
      'COMUNICACI?N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'MATEM?TICAS': 'MATEMATICAS',
      'EL?CTRICA': 'ELECTRICA',
      'M?SICA': 'MUSICA',
      'DISE??O': 'DISENO',
      'COMUNICACI??N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'GESTI?N': 'GESTION',
      'GR??FICA': 'GRAFICA',
      'B?SICA': 'BASICA',
      'ADMINISTRACI?N': 'ADMINISTRACION',
      'P?BLICA': 'PUBLICA',
      'FARMAC?UTICA': 'FARMACEUTICA',
      'FILOSOF?A': 'FILOSOFIA',
      'ESC?NICAS': 'ESCENICAS',
      'ECONOM?A': 'ECONOMIA',
      'GASTRONOM?A': 'GASTRONOMIA',
      'GEOLOG?A': 'GEOLOGIA',
      'LOG?STICA': 'LOGISTICA',
      'ART?STICA': 'ARTISTICA',
      'PEDAGOG?A': 'PEDAGOGIA',
      ' DE ': ' ',
      ' EN ': ' ',
      '  ': ' ',
      '.': '',
      '- ': ' ',
  }
  for wrong, well in wrong_words_dict.items():
    if wrong in result: result = result.replace(wrong, well)
  return result


onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(unidecode.unidecode)
onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(lambda sentence: sentence.upper())
onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(replace_wrong_words)
onehoted_df["E_PRGM_DEPARTAMENTO"] = onehoted_df["E_PRGM_DEPARTAMENTO"].map(unidecode.unidecode)
onehoted_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,7,2,3,6,Si,Si,N,...,0.322,0.208,0.31,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,5,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.19,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212224,20212,INGENIERIA INDUSTRIAL,BOGOTA,5,5,2,5,Si,No,N,...,0.233,0.284,0.298,0.311,0,0,1,0,0,1
212225,20183,INGENIERIA AMBIENTAL,SANTANDER,4,5,3,3,Si,No,N,...,0.315,0.214,0.318,0.267,0,0,1,0,1,0
212226,20203,MEDICINA,QUINDIO,8,5,5,9,Si,Si,N,...,0.529,0.126,0.249,0.187,0,0,1,0,1,0
212227,20195,DERECHO,VALLE,5,5,4,2,Si,Si,N,...,0.469,0.155,0.265,0.193,0,0,1,0,0,1


In [11]:
onehoted_df = onehoted_df.drop(['F_TIENEINTERNET_Miss'], axis=1)
print (onehoted_df.shape)
onehoted_df

(212229, 23)


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,7,2,3,6,Si,Si,N,...,3,0.322,0.208,0.31,0.267,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,1,0.311,0.215,0.292,0.264,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,5,5,3,5,Si,No,N,...,1,0.297,0.214,0.305,0.264,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,4,0.485,0.172,0.252,0.19,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,2,0.316,0.232,0.285,0.294,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212224,20212,INGENIERIA INDUSTRIAL,BOGOTA,5,5,2,5,Si,No,N,...,1,0.233,0.284,0.298,0.311,0,1,0,0,1
212225,20183,INGENIERIA AMBIENTAL,SANTANDER,4,5,3,3,Si,No,N,...,2,0.315,0.214,0.318,0.267,0,1,0,1,0
212226,20203,MEDICINA,QUINDIO,8,5,5,9,Si,Si,N,...,4,0.529,0.126,0.249,0.187,0,1,0,1,0
212227,20195,DERECHO,VALLE,5,5,4,2,Si,Si,N,...,3,0.469,0.155,0.265,0.193,0,1,0,0,1


In [12]:
final_student_df = replace_column_with_onehot(onehoted_df, 'E_PRGM_DEPARTAMENTO')
final_student_df['E_PRGM_ACADEMICO'] = final_student_df['E_PRGM_ACADEMICO'].astype('category').cat.codes
final_student_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,F_TIENECOMPUTADOR,...,E_PRGM_DEPARTAMENTO_NORTE SANTANDER,E_PRGM_DEPARTAMENTO_PUTUMAYO,E_PRGM_DEPARTAMENTO_QUINDIO,E_PRGM_DEPARTAMENTO_RISARALDA,E_PRGM_DEPARTAMENTO_SAN ANDRES,E_PRGM_DEPARTAMENTO_SANTANDER,E_PRGM_DEPARTAMENTO_SUCRE,E_PRGM_DEPARTAMENTO_TOLIMA,E_PRGM_DEPARTAMENTO_VALLE,E_PRGM_DEPARTAMENTO_VAUPES
0,20212,225,7,2,3,6,Si,Si,N,Si,...,0,0,0,0,0,0,0,0,0,0
1,20212,184,5,1,3,7,Si,No,N,Si,...,0,0,0,0,0,0,0,0,0,0
2,20203,588,5,5,3,5,Si,No,N,No,...,0,0,0,0,0,0,0,0,0,0
3,20195,20,6,1,4,1,Si,No,N,Si,...,0,0,0,0,0,1,0,0,0,0
4,20212,661,5,4,3,3,Si,Si,N,Si,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212224,20212,319,5,5,2,5,Si,No,N,No,...,0,0,0,0,0,0,0,0,0,0
212225,20183,286,4,5,3,3,Si,No,N,Si,...,0,0,0,0,0,1,0,0,0,0
212226,20203,579,8,5,5,9,Si,Si,N,Si,...,0,0,1,0,0,0,0,0,0,0
212227,20195,184,5,5,4,2,Si,Si,N,Si,...,0,0,0,0,0,0,0,0,1,0


In [13]:
y = final_student_df["RENDIMIENTO_GLOBAL"].values
X = final_student_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
del final_student_df
print (X.shape, y.shape)

(212229, 52) (212229,)


In [14]:
def rel_mrae(estimator, X, y):
    preds = estimator.predict(X)
    return np.mean(np.abs(preds-y)/y)

def bootstrap_score(estimator, X, y, test_size):
  trscores, tsscores = [], []
  for _ in range(10):
    Xtr, Xts, ytr, yts = train_test_split(X,y, test_size=test_size)
    estimator.fit(Xtr, ytr)
    trscores.append(rel_mrae(estimator, Xtr, ytr))
    tsscores.append(rel_mrae(estimator, Xts, yts))

  return (np.mean(trscores), np.std(trscores)), (np.mean(tsscores), np.std(tsscores))

In [15]:
# Verificar columnas no numéricas
non_numerics = X.select_dtypes(include=['object', 'category']).columns
print("Columnas no numéricas:", non_numerics.tolist())
print(X[non_numerics].nunique())

# Codificar columnas restantes automáticamente
X = pd.get_dummies(X, columns=non_numerics)

Columnas no numéricas: ['E_VALORMATRICULAUNIVERSIDAD', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
E_VALORMATRICULAUNIVERSIDAD     10
F_TIENELAVADORA                  3
F_TIENEAUTOMOVIL                 3
E_PRIVADO_LIBERTAD               3
F_TIENECOMPUTADOR                3
F_TIENEINTERNET.1                3
INDICADOR_1                    376
INDICADOR_2                    343
INDICADOR_3                    197
INDICADOR_4                    277
dtype: int64


In [18]:
# Detectar columnas no numéricas
non_numerics = X.select_dtypes(include=['object', 'category']).columns
X = pd.get_dummies(X, columns=non_numerics)

# Convertir 'Miss' en y a un valor numérico (ej. 0) y asegurar que y sea numérico
y = np.where(y == 'Miss', 0, y).astype(float)

# Entrenamiento
estimator = LinearRegression()
(trmean, trstd), (tsmean, tsstd) = bootstrap_score(estimator, X, y, test_size=0.3)
print("train score %.3f (±%.4f)" % (trmean, trstd))
print("test score  %.3f (±%.4f)" % (tsmean, tsstd))

/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)
/tmp/ipython-input-1244505102.py:3: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs(preds-y)/y)


train score inf (±nan)
test score  inf (±nan)


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:185: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
